This notebook walks you through the main schemas and tables of the IBL behavior pipeline, and show you how to plot psychometric curves with data from the database:

1. Fetch trial data and compute psychometric function yourself
2. Fetch the pre-computed fit results and original data and plot directly.

# Behavior tables

Here is a list of major tables in schema `ibl_behavior`:  

* TrialSet: Trial collection of a behavioral session.
* TrialSet.Trial: data of each individual trial in a TrialSet. Contains all the information in the ALF files `trials.*`

# Behavior analyses tables

Here is a list of major tables in schema `ibl_analyses_behavior`:  

* PsychResults: fit parameters of the entire TrialSet (Session). 
* PsychResultsBlock: fit parameters of each block with different probability left.
* ReactionTime: median reaction time for the entire TrialSet (Session)
* ReactionTimeContrastBlock: median reaction time as a function of contrast, for each block with different probability left.
* BehavioralSummaryByDate: results achieved with all trials of the day
>* PsychResults: fit parameters of each probability left. 
>* ReactionTimeContrast: For each probability left, reaction time of each contrast.
>* ReactionTimeByDate: median reaction time of the date.
* TrainingStatus: look up table for available status, 'untrainable', 'unbiasable', 'in_training', 'trained_1a', 'trained_1b', 'ready4ephysrig', 'ready4delay', 'ready4recording' 
* SessionTrainingStatus: training status of each session


# Plot Psychometric curves

Here we first import some modules from the ibl pipeline and some other modules

In [ ]:
# behavior data
from ibl_pipeline import behavior

# analyzed result of behavioral data
from ibl_pipeline.analyses import behavior as behavior_analyses

# meta information of the subject and session
from ibl_pipeline import subject, acquisition

# Function to perform the model fits of the psychometric function
from ibl_pipeline.utils import psychofit as psy

# some regular modules
import numpy as np
import datetime
import seaborn as sns 
import matplotlib.pyplot as plt

In [ ]:
%matplotlib notebook

## Fetch data from behavior.TrialSet.Trial table and compute psych curve yourself

Let's take a quick look at the table `behavior.TrialSet`

In [ ]:
behavior.TrialSet()

The TrialSet table stores some summary statistics in one session of behavioral experiment. To also show the information of the subject, we could join the table with subject related tables.

In [ ]:
behavior.TrialSet * subject.Subject * subject.SubjectLab * subject.SubjectProject

We could restrict to one session by:

In [ ]:
import datetime
q = behavior.TrialSet * subject.Subject * subject.SubjectLab * subject.SubjectProject & \
    'subject_nickname="CSHL_015"' & {'session_start_time': datetime.datetime(2019, 9, 16, 13, 44, 46)}
q

The trial-by-trial information are shown in a **part table** `behavior.TrialSet.Trial`

We could check the documentation of each of the column with `describe()`

In [ ]:
behavior.TrialSet.Trial.describe();

To fetch some part of data, we could do use the fetch method:  

For example we would like to see the stimulus contrasts, and the animal choices in one session (that we already queried and saved the results as q), we could do:

In [ ]:
# we could fetch the fields as a list of dictionary, only for the trials with a choice
data = (behavior.TrialSet.Trial & q & 'trial_response_choice !="No Go"').fetch(
    'trial_stim_contrast_left', 'trial_stim_contrast_right', 'trial_response_choice', as_dict=True)

In [ ]:
# We could then convert the data to a dataframe for further analyses
import pandas as pd
df = pd.DataFrame(data)
df

Then we can do some computation here to explore the data.

We first compute the signed contrast, so that the contrasts on the right are positive

In [ ]:
df['signed_contrast'] = df['trial_stim_contrast_right'] - df['trial_stim_contrast_left']

In [ ]:
df['report_right'] = df['trial_response_choice'] == "CCW"

In [ ]:
report_right = df.groupby(['signed_contrast'], as_index=False).mean()
report_right

Then let's plot the psychometric curve: prob_report_right vs signed_contrast:

In [ ]:
plt.plot(report_right['signed_contrast'], report_right['report_right'], 'o')
plt.xlabel('Signed Contrast')
plt.ylabel('Probability reporting right')

## Plot Psychometric curve with pre-computed tables

Alternatively, we could plot the psychometric curves with results directly fetched from the pre-computed tables. The fit results of the psychometric curve are saved in the table `behavior_analyses.PsychResultsBlock`, we can browse entries in the table for a particular subject, the prob_left_block marks which prior of probability left have been used in the block

In [ ]:
q = behavior_analyses.PsychResultsBlock & (subject.Subject & 'subject_nickname="CSHL_015"')

Let's check the results of sessions collected after 2019-09-15, by:

In [ ]:
q & 'session_start_time > "2019-09-15"'

Let's plot the psychometric curve of the last session on 9/17, containing three blocks with `prob_left` 0.2, 0.5 and 0.8

In [ ]:
psych_results = q & {'session_start_time': datetime.datetime(2019, 9, 16, 13, 44, 46)}

Now let's fetch the fit parameters and behavioral statistics from the table as a list of dictionaries.

In [ ]:
dict_results = psych_results.fetch(
    'signed_contrasts', 'prob_choose_right', 'n_trials_stim', 'n_trials_stim_right',
    'threshold', 'bias', 'lapse_low', 'lapse_high', as_dict=True)

Next let's loop through the blocks and plot the psychometric curves:

In [ ]:
import warnings
warnings.filterwarnings('ignore')
colors = [[1., 0., 0.], [0., 0., 0.], [0., 0., 1.]]

fig, ax = plt.subplots()
for result, color in zip(dict_results, colors):
    pars = [result['bias'], result['threshold'], result['lapse_low'], result['lapse_high']]
    contrasts = result['signed_contrasts'] * 100
    contrasts_fit = np.arange(-100, 100)
    prob_right_fit = psy.erf_psycho_2gammas(pars, contrasts_fit) *100
    sns.lineplot(contrasts_fit, prob_right_fit, color=color, ax=ax)
    sns.lineplot(x=contrasts, y=result['prob_choose_right']*100, err_style="bars", linewidth=0, linestyle='None', mew=0.5,
            marker='.', ci=68, color=color, ax=ax)

plt.gca().set_xlabel('Signed Contrast (%)')
plt.gca().set_ylabel('Rightward Choice (%)')
